<a href="https://colab.research.google.com/github/akapo/class-llm/blob/main/chapter2/langchain_step2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 실습 환경 설정

**필요한 패키지 설치**

In [ ]:
!pip install -qU openai
!pip install -qU langchain langchain-openai langchain-community langchainhub langchain-experimental
!pip install -qU huggingface_hub langchain-google-genai langchain-anthropic
!pip install -Uq sentence-transformers
!pip install -qU python-dotenv
!pip install -qU pymupdf pypdf
!pip install -qU numexpr
!pip install -qU faiss-cpu
!pip install -qU chromadb
!pip install -qU bs4
!pip install -qU google-search-results duckduckgo-search wikipedia
#!pip install -qU streamlit
#!pip install -qU streamlit-chat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.3/377.3 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/4

In [ ]:
#!pip install python-dotenv
import dotenv
dotenv.load_dotenv()

import os
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
ANTHROPIC_API_KEY = os.getenv('ANTHROPIC_API_KEY')

# 주의: API_KEY를 깃허브에 노출하면 LLM 업체는 해당키를 즉시 삭제함.
print("GOOGLE_API_KEY: " + GOOGLE_API_KEY[:20])
print("OPENAI_API_KEY: " + OPENAI_API_KEY[:20])
print("ANTHROPIC_API_KEY: " + ANTHROPIC_API_KEY[:20])

GOOGLE_API_KEY: AIzaSyCCtFS9bml5jdGI
OPENAI_API_KEY: sk-proj-FEDi6Ge6iQyR
ANTHROPIC_API_KEY: sk-ant-api03-YcGqyQQ


# **[LangChain]　3. Memory**

**기억력을 구현하지 않았을 때 예시**

In [ ]:
# LLM은 붕어인가? 왜 아무것도 기억을 못하는가?
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

while True:
    user_message = input("You: ")
    message = HumanMessage(content=user_message)

    if user_message == "끝":
        print("(대화 종료)")
        break

    ai_message = llm.invoke([message])
    print(f"AI: {ai_message.content}")

You: 안녕? 나는 정진이야.
AI: 안녕하세요, 정진님! 어떻게 도와드릴까요?
You: 내 이름 기억해?
AI: 죄송하지만, 저는 개인적인 정보를 기억할 수 없습니다. 하지만 도움이 필요하시면 언제든지 말씀해 주세요!
You: 끝
(대화 종료)


**기억력 직접 구현**

In [ ]:
# 메시지 리스트로 기억력 직접구현
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

message = []
while True:
    user_message = input("You: ")
    message.append(HumanMessage(content=user_message))

    if user_message == "끝":
        print("(대화 종료)")
        break

    response = llm.invoke(message).content
    print(f"AI: {response}")
    ai_message = AIMessage(content=response)
    message.append(ai_message)

You: 안녕? 나는 정진이야.
AI: 안녕하세요, 정진님! 어떻게 도와드릴까요?
You: 내 이름 기억해?
AI: 네, 정진님! 당신의 이름을 기억하고 있습니다. 어떻게 도와드릴까요?
You: 끝
(대화 종료)


**ConversationBufferMemory 예제1**

In [ ]:
# ConversationBufferMemory를 이용한 기억력 구현
from langchain.chains import ConversationChain
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
conversation = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory(),
)

while True:
    user_message = input("You: ")

    if user_message == "끝":
        print("(대화 종료)")
        break

    ai_message = conversation.invoke(input=user_message)["response"]
    print(f"AI: {ai_message}")

You: 안녕? 나는 정진이야.
AI: 안녕, 정진! 만나서 반가워! 나는 AI야. 오늘 기분이 어때? 혹시 특별한 계획이 있니?
You: 아직 계획은 없어... 너 내 이름 기억해?
AI: 네, 정진! 너의 이름을 기억하고 있어. 앞으로도 계속 기억할게. 혹시 어떤 취미나 관심사가 있어? 함께 이야기해보면 좋을 것 같아!
You: 끝
(대화 종료)


**ConversationBufferMemory 예제2**

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import ConversationChain

# 1) 시스템 설정: 역할부여 정의
system_template = SystemMessagePromptTemplate.from_template("너는 지금 사람과 '1 to 50 게임'을 하는 AI야.")

# 2) ChatPromptTemplate 템플릿 정의
prompt = ChatPromptTemplate.from_messages([
    system_template,                                       # 역할부여
    MessagesPlaceholder(variable_name="chat_history"),     # 메모리 저장소 설정. ConversationBufferMemory의 memory_key 와 동일하게 설정
    HumanMessagePromptTemplate.from_template("{input}"),   # 사용자 메시지 injection
])

# 3) 메모리 정의
# `return_messages=True`를 사용해야 MessagesPlaceholder에 주입됨
# `chat_history`가 MessagesPlaceholder 이름과 일치해야함
memory = ConversationBufferMemory(memory_key="chat_history",
                                  ai_prefix="AI",
                                  human_prefix="Human",
                                  return_messages=True)

# 4) LLM 모델 정의
llm = ChatOpenAI(model_name='gpt-4o-mini')

# 5) LLMChain 정의
conversation = ConversationChain(
    llm=llm,       # LLM
    prompt=prompt, # Prompt
    memory=memory, # 메모리
    verbose=True,  # True 로 설정시 로그 출력
)

user_message = "내가 생각한 숫자가 무엇인지 맞춰봐"
while True:
    ai_message = conversation.invoke(input=user_message)["response"]
    print(f"AI: {ai_message}")

    user_message = input("You: ")
    if user_message == "끝":
        print("(대화 종료)")
        break



> Entering new ConversationChain chain...
Prompt after formatting:
System: 너는 지금 사람과 '1 to 50 게임'을 하는 AI야.
Human: 내가 생각한 숫자가 무엇인지 맞춰봐

> Finished chain.
AI: 좋아요! 1부터 50 사이의 숫자를 맞춰볼게요. 제 첫 번째 추측은 25입니다. 맞혔나요?
You: 보다커


> Entering new ConversationChain chain...
Prompt after formatting:
System: 너는 지금 사람과 '1 to 50 게임'을 하는 AI야.
Human: 내가 생각한 숫자가 무엇인지 맞춰봐
AI: 좋아요! 1부터 50 사이의 숫자를 맞춰볼게요. 제 첫 번째 추측은 25입니다. 맞혔나요?
Human: 보다커

> Finished chain.
AI: 알겠어요! 그럼 제 다음 추측은 38입니다. 이 숫자는 어떤가요?
You: 보다 작아


> Entering new ConversationChain chain...
Prompt after formatting:
System: 너는 지금 사람과 '1 to 50 게임'을 하는 AI야.
Human: 내가 생각한 숫자가 무엇인지 맞춰봐
AI: 좋아요! 1부터 50 사이의 숫자를 맞춰볼게요. 제 첫 번째 추측은 25입니다. 맞혔나요?
Human: 보다커
AI: 알겠어요! 그럼 제 다음 추측은 38입니다. 이 숫자는 어떤가요?
Human: 보다 작아

> Finished chain.
AI: 좋아요! 그럼 32는 어떤가요?
You: 정답이야


> Entering new ConversationChain chain...
Prompt after formatting:
System: 너는 지금 사람과 '1 to 50 게임'을 하는 AI야.
Human: 내가 생각한 숫자가 무엇인지 맞춰봐
AI: 좋아요! 1부터 50 사이의 숫자를 맞춰볼게요. 제 첫 번째 추측은 25입니다. 맞혔나요?
Human: 보다커

# **[LangChain]　4. Retrival**

## **1) Loader & Splitter**

**CharacterTextSplitter**

In [ ]:
# TextLoader & RecursiveCharacterTextSplitter
# Playground: https://chunkviz.up.railway.app/
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

splitter = CharacterTextSplitter(
    separator = "",   # <= 실험
    chunk_size=300,
    chunk_overlap=30,
    length_function=len,
)
document = TextLoader("appendix-keywords.txt").load()
split_docs = splitter.split_documents(document)

# 총 분할된 도큐먼트 수
print(f'총 분할된 도큐먼트 수: {len(split_docs)}')
print(split_docs[0].page_content)
print('======')
print(split_docs[1].page_content)
print('======')
print(split_docs[2].page_content)

총 분할된 도큐먼트 수: 22
Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding

정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.
예시: "사과"라는 단
해하고 처리할 수 있게 합니다.
예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.
연관키워드: 자연어 처리, 벡터화, 딥러닝

Token

정의: 토큰은 텍스트를 더 작은 단위로 분할하는 것을 의미합니다. 이는 일반적으로 단어, 문장, 또는 구절일 수 있습니다.
예시: 문장 "나는 학교에 간다"를 "나는", "학교에", "간다"로 분할합니다.
연관키워드: 토큰화, 자연어 처리, 구문 분석

Tokenizer

정의: 토크나이저는 텍스트 데이터를 토큰으로 분할하는 도구입니다. 이는 자연어 처리에
터를 토큰으로 분할하는 도구입니다. 이는 자연어 처리에서 데이터를 전처리하는 데 사용됩니다.
예시: "I love programming."이라는 문장을 ["I", "love", "programming", "."]으로 분할합니다.
연관키워드: 토큰화, 자연어 처리, 구문 분석

VectorStore

정의: 벡터스토어는 벡터 형식으로 변환된 데이터를 저장하는 시스템입니다. 이는 검색, 분류 및 기타 데이터 분석 작업에 사용됩니다.
예시: 단어 임베딩 벡터들을 데이터베이스에 저장하여 빠르게 접근할 수 있습니다.
연관키워드: 임베딩, 데


**RecursiveCharacterTextSplitter**

In [ ]:
# TextLoader & RecursiveCharacterTextSplitter
# Playground: https://langchain-text-splitter.streamlit.app/
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, Language

splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=30,
    length_function=len,
)
document = TextLoader("appendix-keywords.txt").load()
split_docs = splitter.split_documents(document)

# 총 분할된 도큐먼트 수
print(f'총 분할된 도큐먼트 수: {len(split_docs)}')
print(split_docs[0].page_content)
print('======')
print(split_docs[1].page_content)
print('======')
print(split_docs[2].page_content)

총 분할된 도큐먼트 수: 30
Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding
Embedding

정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.
예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.
연관키워드: 자연어 처리, 벡터화, 딥러닝

Token
Token

정의: 토큰은 텍스트를 더 작은 단위로 분할하는 것을 의미합니다. 이는 일반적으로 단어, 문장, 또는 구절일 수 있습니다.
예시: 문장 "나는 학교에 간다"를 "나는", "학교에", "간다"로 분할합니다.
연관키워드: 토큰화, 자연어 처리, 구문 분석

Tokenizer


**PyPDFLoader**

In [ ]:
# PDFLoader
#!pip install -q pypdf
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("소나기 - 황순원.pdf")
docs = loader.load()

print(f'총 분할된 도큐먼트 수: {len(docs)}')
docs[0].page_content

총 분할된 도큐먼트 수: 7


'- 1 -소나기\n황순원\n소년은 개울가에서 소녀를 보자 곧 윤 초시네 증손녀 (曾孫女 )딸이라는 걸 알 수 있었다 . \n소녀는 개울에다 손을 잠그고 물장난을 하고 있는 것이다 . 서울서는 이런 개울물을 보지 \n못하기나 한 듯이.\n벌써 며칠째 소녀는 , 학교에서 돌아오는 길에 물장난이었다 . 그런데 , 어제까지 개울 기슭에\n서 하더니 , 오늘은 징검다리 한가운데 앉아서 하고 있다.\n소년은 개울둑에 앉아 버렸다 . 소녀가 비키기를 기다리자는 것이다 .\n요행 지나가는 사람이 있어, 소녀가 길을 비켜 주었다 .\n다음 날은 좀 늦게 개울가로 나왔다 .\n이 날은 소녀가 징검다리 한가운데 앉아 세수를 하고 있었다 . 분홍 스웨터 소매를 걷어올\n린 목덜미가 마냥 희었다 .\n한참 세수를 하고 나더니 , 이번에는 물 속을 빤히 들여다 본다. 얼굴이라도 비추어 보는 \n것이리라 . 갑자기 물을 움켜 낸다. 고기 새끼라도 지나가는 듯.\n소녀는 소년이 개울둑에 앉아 있는 걸 아는지 모르는지 그냥 날쌔게 물만 움켜 낸다. 그러\n나, 번번이 허탕이다 . 그대로 재미있는 양, 자꾸 물만 움킨다 . 어제처럼 개울을 건너는 사\n람이 있어야 길을 비킬 모양이다 .\n그러다가 소녀가 물 속에서 무엇을 하나 집어 낸다. 하얀 조약돌이었다 . 그리고는 벌떡 일\n어나 팔짝팔짝 징검다리를 뛰어 건너간다 .\n다 건너가더니만 홱 이리로 돌아서며 ,\n“이 바보.”\n조약돌이 날아왔다 .\n소년은 저도 모르게 벌떡 일어섰다 .\n단발 머리를 나풀거리며 소녀가 막 달린다 . 갈밭 사잇길로 들어섰다 . 뒤에는 청량한 가을 \n햇살 아래 빛나는 갈꽃뿐 .\n이제 저쯤 갈밭머리로 소녀가 나타나리라 . 꽤 오랜 시간이 지났다고 생각됐다 . 그런데도 \n소녀는 나타나지 않는다 . 발돋움을 했다. 그러고도 상당한 시간이 지났다고 생각됐다 .\n저 쪽 갈밭머리에 갈꽃이 한 옴큼 움직였다 . 소녀가 갈꽃을 안고 있었다 . 그리고 , 이제는 \n천천한 걸음이었다 . 유난히 맑은 가을 햇살이 소녀

**PyMuPDFLoader**

In [ ]:
# PyMuPDFLoader
#!pip install -q pymypdf
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("소나기 - 황순원.pdf")
docs = loader.load()

print(f'총 분할된 도큐먼트 수: {len(docs)}')
docs[0].page_content

총 분할된 도큐먼트 수: 7


'- 1 -\n소나기\n황순원\n소년은 개울가에서 소녀를 보자 곧 윤 초시네 증손녀(曾孫女)딸이라는 걸 알 수 있었다. \n소녀는 개울에다 손을 잠그고 물장난을 하고 있는 것이다. 서울서는 이런 개울물을 보지 \n못하기나 한 듯이.\n벌써 며칠째 소녀는, 학교에서 돌아오는 길에 물장난이었다. 그런데, 어제까지 개울 기슭에\n서 하더니, 오늘은 징검다리 한가운데 앉아서 하고 있다.\n소년은 개울둑에 앉아 버렸다. 소녀가 비키기를 기다리자는 것이다.\n요행 지나가는 사람이 있어, 소녀가 길을 비켜 주었다.\n다음 날은 좀 늦게 개울가로 나왔다.\n이 날은 소녀가 징검다리 한가운데 앉아 세수를 하고 있었다. 분홍 스웨터 소매를 걷어올\n린 목덜미가 마냥 희었다.\n한참 세수를 하고 나더니, 이번에는 물 속을 빤히 들여다 본다. 얼굴이라도 비추어 보는 \n것이리라. 갑자기 물을 움켜 낸다. 고기 새끼라도 지나가는 듯.\n소녀는 소년이 개울둑에 앉아 있는 걸 아는지 모르는지 그냥 날쌔게 물만 움켜 낸다. 그러\n나, 번번이 허탕이다. 그대로 재미있는 양, 자꾸 물만 움킨다. 어제처럼 개울을 건너는 사\n람이 있어야 길을 비킬 모양이다.\n그러다가 소녀가 물 속에서 무엇을 하나 집어 낸다. 하얀 조약돌이었다. 그리고는 벌떡 일\n어나 팔짝팔짝 징검다리를 뛰어 건너간다.\n다 건너가더니만 홱 이리로 돌아서며,\n“이 바보.”\n조약돌이 날아왔다.\n소년은 저도 모르게 벌떡 일어섰다.\n단발 머리를 나풀거리며 소녀가 막 달린다. 갈밭 사잇길로 들어섰다. 뒤에는 청량한 가을 \n햇살 아래 빛나는 갈꽃뿐.\n이제 저쯤 갈밭머리로 소녀가 나타나리라. 꽤 오랜 시간이 지났다고 생각됐다. 그런데도 \n소녀는 나타나지 않는다. 발돋움을 했다. 그러고도 상당한 시간이 지났다고 생각됐다.\n저 쪽 갈밭머리에 갈꽃이 한 옴큼 움직였다. 소녀가 갈꽃을 안고 있었다. 그리고, 이제는 \n천천한 걸음이었다. 유난히 맑은 가을 햇살이 소녀의 갈꽃머리에서 반짝거렸다. 소녀 아닌 \n갈꽃이 들길을 걸

## **2) Embeddings**

In [ ]:
# OpenAI Embedding 모델
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

text="만나서 반가워요."
embeded = embeddings.embed_query(text)
print(str(len(embeded))+'차원')
print(embeded[:5])

1536차원
[-0.019351087510585785, -0.03914021700620651, 0.00637413514778018, -0.01435227133333683, -0.009520941413939]


In [ ]:
# 코사인 유사도 계산
import numpy as np
from numpy import dot
from numpy.linalg import norm

# 벡터 A와 벡터 B의 코사인 유사도 계산
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [ ]:
# OpenAI Embeddings 모델 테스트
from langchain_openai import OpenAIEmbeddings
embeddings_model = OpenAIEmbeddings()

embeddings = embeddings_model.embed_documents(
    [
        '안녕하세요!',
        '어! 오랜만이에요',
        '이름이 어떻게 되세요?',
        '날씨가 추워요',
        'Hello LLM!'
    ]
)

embedded_query = embeddings_model.embed_query('첫인사를 하고 이름을 물어봤나요?')
print(str(len(embedded_query))+'차원')

for embedding in embeddings:
    print(cos_sim(embedding, embedded_query))

1536차원
0.8348159356519259
0.8153598883956
0.8843727343154585
0.7898927808309193
0.7469070089071632


In [ ]:
# HuggingFace의 jhgan/ko-sroberta-nli 모델 테스트
#!pip install -Uq sentence-transformers
from langchain.embeddings import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(
    model_name='jhgan/ko-sroberta-nli',
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True},
)

embeddings = embeddings_model.embed_documents(
    [
        '안녕하세요!',
        '어! 오랜만이에요',
        '이름이 어떻게 되세요?',
        '날씨가 추워요',
        'Hello LLM!'
    ]
)

embedded_query = embeddings_model.embed_query('첫인사를 하고 이름을 물어봤나요?')
print(str(len(embedded_query))+'차원')

for embedding in embeddings:
    print(cos_sim(embedding, embedded_query))

768차원
0.5899016189601531
0.4182631225980652
0.7240604521610333
0.05702662997392148
0.4316418328113528


In [ ]:
# GoogleGenerativeAIEmbeddings 모델 테스트
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings_model = GoogleGenerativeAIEmbeddings(model='models/embedding-001')

embeddings = embeddings_model.embed_documents(
    [
        '안녕하세요!',
        '어! 오랜만이에요',
        '이름이 어떻게 되세요?',
        '날씨가 추워요',
        'Hello LLM!'
    ]
)

embedded_query = embeddings_model.embed_query('첫인사를 하고 이름을 물어봤나요?')
print(str(len(embedded_query))+'차원')

for embedding in embeddings:
    print(cos_sim(embedding, embedded_query))

768차원
0.6051636939358793
0.620231830582371
0.7160855778483425
0.6574387494383126
0.55527388841408


In [ ]:
# Google GenAI의 Embedding 모델
# 텍스트를 특정 너비로 감싸는 데 사용되는 라이브러리를 가져옵니다.
import textwrap
# 과학 계산을 위한 라이브러리인 numpy를 가져옵니다.
import numpy as np
# 데이터 분석을 위한 라이브러리인 pandas를 가져옵니다.
import pandas as pd
# Google의 생성적 AI 모듈을 가져옵니다.
import google.generativeai as genai
# Google의 생성적 언어 모델을 가져옵니다.
import google.ai.generativelanguage as glm
# IPython 환경에서 Markdown 형식의 텍스트를 표시하기 위한 모듈을 가져옵니다.
from IPython.display import Markdown
# genai 라이브러리의 list_models 함수를 사용하여 사용 가능한 모델들의 목록을 가져옵니다.

GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

for m in genai.list_models():
    # 각 모델의 supported_generation_methods 속성 중 'embedContent' 메소드를 지원하는지 확인합니다.
    if "embedContent" in m.supported_generation_methods:
        # 'embedContent' 메소드를 지원하는 모델의 이름을 출력합니다.
        print(m.name)

# 제목을 문자열로 선언합니다.
title = "The next generation of AI for developers and Google Workspace"
# 샘플 텍스트를 문자열로 선언합니다. 여기에는 제목과 본문이 포함되어 있습니다.
sample_text = (
    "Title: The next generation of AI for developers and Google Workspace"
    "\n"
    "Full article:\n"
    "\n"
    "Gemini API & Google AI Studio: An approachable way to explore and prototype with generative AI applications"
)

# 사용할 모델의 이름을 문자열로 선언합니다.
model = "models/embedding-001"
# genai.embed_content 함수를 사용하여 콘텐츠의 임베딩을 생성합니다.
# 이 함수는 모델, 콘텐츠, 작업 유형, 제목을 인자로 받습니다.
embeded = genai.embed_content(
    model=model, content=sample_text, task_type="retrieval_document", title=title
)

# 생성된 임베딩을 출력합니다.
print(str(len(embeded['embedding']))+'차원')
print(embeded)


text="진희는 강아지를 키우고 있습니다. 진희가 키우고 있는 동물은?"
# genai.embed_content 함수를 사용하여 콘텐츠의 임베딩을 생성합니다.
# 이 함수는 모델, 콘텐츠, 작업 유형, 제목을 인자로 받습니다.
embeded = genai.embed_content(
    model=model, content=text
)

# 생성된 임베딩을 출력합니다.
print(str(len(embeded['embedding']))+'차원')
print(embeded)

models/embedding-001
models/text-embedding-004
768차원
{'embedding': [0.03411343, -0.05517662, -0.020209055, -0.0041249567, 0.058917783, 0.014129515, 0.0045353593, 0.0014303668, 0.05976634, 0.08292115, 0.007162964, 0.0069041685, -0.053083427, -0.010905125, 0.0321402, -0.037163995, 0.050372455, 0.019348344, -0.037328612, 0.026647927, 0.030781753, -0.011288501, -0.031485256, -0.060248993, -0.026219442, -0.009794235, 0.006630139, -0.01846516, -0.026324715, 0.020442624, -0.06317684, 0.014559574, -0.052296035, 0.016451128, -9.720217e-05, -0.051706687, -0.0054406044, -0.056967627, 0.011144145, -0.009201792, -0.0021951047, -0.1099701, -0.011712193, 0.021221714, 0.009171804, -0.029621972, 0.034534883, 0.039578073, 0.019021519, -0.06269169, 0.039473332, 0.052403256, 0.061814185, -0.034507945, -0.009557816, -0.0049551064, 0.017839009, -0.021176832, 0.015043588, 0.015390569, -0.006334281, 0.043696404, -0.028341983, 0.028433999, 0.01472686, -0.06585564, -0.044533554, 0.0055523133, 0.035775978, 0.031

## **3) Vector Store**

**[벡터저장소]** FAISS(Facebook AI Similarity Search)

In [ ]:
# FAISS 유사도 기반 검색
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, Language
from langchain.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_openai import OpenAIEmbeddings

splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=40,
    length_function=len,
)

embeddings_model = OpenAIEmbeddings()

document = TextLoader("appendix-keywords.txt").load()
split_docs = splitter.split_documents(document)
print(f'총 분할된 도큐먼트 수: {len(split_docs)}')

#vector store
vs = FAISS.from_documents(
    split_docs,
    embedding = embeddings_model,
    #유사도 검색방식 지정
    #distance_strategy = DistanceStrategy.EUCLIDEAN_DISTANCE
    #distance_strategy = DistanceStrategy.COSINE
)
print(f"{vs.distance_strategy}\n")

query = "FAISS가 뭐야?"
docs = vs.similarity_search(query, k=2)
for index, doc in enumerate(docs, start=1):
    print(f'[{index}]\n' + doc.page_content+ "\n========\n")

query = "GPT란?"
docs = vs.similarity_search(query, k=2)
for index, doc in enumerate(docs, start=1):
    print(f'[{index}]\n' + doc.page_content+ "\n========\n")

총 분할된 도큐먼트 수: 19
EUCLIDEAN_DISTANCE

[1]
FAISS (Facebook AI Similarity Search)

정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.
예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.
연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화

Open Source

정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용, 수정, 배포할 수 있는 소프트웨어를 의미합니다. 이는 협업과 혁신을 촉진하는 데 중요한 역할을 합니다.
예시: 리눅스 운영 체제는 대표적인 오픈 소스 프로젝트입니다.
연관키워드: 소프트웨어 개발, 커뮤니티, 기술 협업

[2]
HuggingFace

정의: HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모델과 도구를 제공하는 라이브러리입니다. 이는 연구자와 개발자들이 쉽게 NLP 작업을 수행할 수 있도록 돕습니다.
예시: HuggingFace의 Transformers 라이브러리를 사용하여 감정 분석, 텍스트 생성 등의 작업을 수행할 수 있습니다.
연관키워드: 자연어 처리, 딥러닝, 라이브러리

Digital Transformation

[1]
정의: GPT는 대규모의 데이터셋으로 사전 훈련된 생성적 언어 모델로, 다양한 텍스트 기반 작업에 활용됩니다. 이는 입력된 텍스트에 기반하여 자연스러운 언어를 생성할 수 있습니다.
예시: 사용자가 제공한 질문에 대해 자세한 답변을 생성하는 챗봇은 GPT 모델을 사용할 수 있습니다.
연관키워드: 자연어 처리, 텍스트 생성, 딥러닝

InstructGPT

[2]
InstructGPT

정의: InstructGPT는 사용자의 지시에 따라 특정한 작업을 수행하기 위해 최적화된 GPT 모델입니다. 이 모델은 보다 정확하고 관련성 높은 결과를 생성하도록 설계되었습니다.
예시:

In [ ]:
# FAISS MMR(Maximum marginal relevances search) 검색
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, Language
from langchain.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_openai import OpenAIEmbeddings

splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=40,
    length_function=len,
)

embeddings_model = OpenAIEmbeddings()

document = TextLoader("appendix-keywords.txt").load()
split_docs = splitter.split_documents(document)
print(f'총 분할된 도큐먼트 수: {len(split_docs)}')

#vector store
vs = FAISS.from_documents(
    split_docs,
    embedding = embeddings_model,
    #distance_strategy = DistanceStrategy.EUCLIDEAN_DISTANCE
    distance_strategy = DistanceStrategy.COSINE
)
print(f"{vs.distance_strategy}\n")

query = "FAISS가 뭐야?"
mmr_docs = vs.max_marginal_relevance_search(query, k=2, fetch_k=5)
for index, doc in enumerate(mmr_docs, start=1):
    print(f'[{index}]\n' + doc.page_content+ "\n========\n")

query = "GPT란?"
mmr_docs = vs.max_marginal_relevance_search(query, k=2, fetch_k=5)
for index, doc in enumerate(mmr_docs, start=1):
    print(f'[{index}]\n' + doc.page_content+ "\n========\n")

총 분할된 도큐먼트 수: 19
COSINE

[1]
FAISS (Facebook AI Similarity Search)

정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.
예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.
연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화

Open Source

정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용, 수정, 배포할 수 있는 소프트웨어를 의미합니다. 이는 협업과 혁신을 촉진하는 데 중요한 역할을 합니다.
예시: 리눅스 운영 체제는 대표적인 오픈 소스 프로젝트입니다.
연관키워드: 소프트웨어 개발, 커뮤니티, 기술 협업

[2]
Schema

정의: 스키마는 데이터베이스나 파일의 구조를 정의하는 것으로, 데이터가 어떻게 저장되고 조직되는지에 대한 청사진을 제공합니다.
예시: 관계형 데이터베이스의 테이블 스키마는 열 이름, 데이터 타입, 키 제약 조건 등을 정의합니다.
연관키워드: 데이터베이스, 데이터 모델링, 데이터 관리

DataFrame

정의: DataFrame은 행과 열로 이루어진 테이블 형태의 데이터 구조로, 주로 데이터 분석 및 처리에 사용됩니다.
예시: 판다스 라이브러리에서 DataFrame은 다양한 데이터 타입의 열을 가질 수 있으며, 데이터 조작과 분석을 용이하게 합니다.
연관키워드: 데이터 분석, 판다스, 데이터 처리

Attention 메커니즘

[1]
정의: GPT는 대규모의 데이터셋으로 사전 훈련된 생성적 언어 모델로, 다양한 텍스트 기반 작업에 활용됩니다. 이는 입력된 텍스트에 기반하여 자연스러운 언어를 생성할 수 있습니다.
예시: 사용자가 제공한 질문에 대해 자세한 답변을 생성하는 챗봇은 GPT 모델을 사용할 수 있습니다.
연관키워드: 자연어 처리, 텍스트 생성, 딥러닝

InstructGPT

[2]
Token

정

**[벡터저장소]** FAISS 로컬 DB Save & Load

In [ ]:
# 벡터저장소(FAISS)
# 임베딩 결과를 파일로 저장해 놓으면 다시 임베딩 할 필요 없이 기존 결과를 불러오면 된다.
#!pip install -q faiss-cpu
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, Language
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings()
faiss_dir = "faiss"
doc_file_path = "appendix-keywords.txt"

if(os.path.isdir(faiss_dir) == False):
    document = TextLoader(doc_file_path).load()

    splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n", "\n", " ", ""],
        chunk_size=400,
        chunk_overlap=40,
        length_function=len,
    )
    split_docs = splitter.split_documents(document)
    print(f'총 분할된 도큐먼트 수: {len(split_docs)}')

    vs = FAISS.from_documents(split_docs, embedding_model)
    vs.save_local(faiss_dir)
    print("saved")
else:
    vs = FAISS.load_local(faiss_dir, embedding_model, allow_dangerous_deserialization=True)
    print('loaded')

query = "FAISS가 뭐야?"
#query = "Pandas란?"
docs = vs.similarity_search(query)
print(docs[0].page_content)

loaded
FAISS (Facebook AI Similarity Search)

정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.
예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.
연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화

Open Source

정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용, 수정, 배포할 수 있는 소프트웨어를 의미합니다. 이는 협업과 혁신을 촉진하는 데 중요한 역할을 합니다.
예시: 리눅스 운영 체제는 대표적인 오픈 소스 프로젝트입니다.
연관키워드: 소프트웨어 개발, 커뮤니티, 기술 협업


In [ ]:
# 벡터저장소(FAISS)
# 임베딩 결과를 파일로 저장해 놓으면 다시 임베딩 할 필요 없이 기존 결과를 불러오면 된다.
#!pip install -q faiss-cpu
from langchain.document_loaders import TextLoader, PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings()
faiss_dir = "faiss"
doc_file_path = "소나기 - 황순원.pdf"

if(os.path.isdir(faiss_dir) == False):
    document = PyMuPDFLoader(doc_file_path).load()

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
    )
    split_docs = splitter.split_documents(document)
    print(f'총 분할된 도큐먼트 수: {len(split_docs)}')

    vs = FAISS.from_documents(split_docs, embedding_model)
    vs.save_local(faiss_dir)
    print("saved")
else:
    vs = FAISS.load_local(faiss_dir, embedding_model, allow_dangerous_deserialization=True)
    print('loaded')

print(f"{vs.distance_strategy}\n")

query = "소녀는 왜 이사를 오게 되었는가?"
docs = vs.max_marginal_relevance_search(query, k=2)
for index, doc in enumerate(docs, start=1):
    print(f'[{index}]' + doc.page_content+ "\n========\n")

query = "소녀가 대추를 딴 이유는?"
docs = vs.max_marginal_relevance_search(query, k=3)
for index, doc in enumerate(docs, start=1):
    print(f'[{index}]' + doc.page_content+ "\n========\n")

# 고찰1
query = "소설 끝에서 소녀는 어떠한 결말을 맞이한거야?"
docs = vs.max_marginal_relevance_search(query, k=6)
for index, doc in enumerate(docs, start=1):
    print(f'[{index}]' + doc.page_content+ "\n========\n")

# 고찰2
query = "소설의 주인공은?"
docs = vs.max_marginal_relevance_search(query, k=3)
for index, doc in enumerate(docs, start=1):
    print(f'[{index}]' + doc.page_content+ "\n========\n")

loaded
EUCLIDEAN_DISTANCE

[1]- 6 -
소년은 소녀네가 이사해 오기 전에 벌써 어른들의 이야기를 들어서, 윤 초시 손자(孫子)가 
서울서 사업에 실패해 가지고 고향에 돌아오지 않을 수 없게 되었다는 걸 알고 있었다. 그
것이 이번에는 고향 집마저 남의 손에 넘기게 된 모양이었다.
“왜 그런지 난 이사 가는 게 싫어졌다. 어른들이 하는 일이니 어쩔 수 없지만…….”
전에 없이, 소녀의 까만 눈에 쓸쓸한 빛이 떠돌았다.
소녀와 헤어져 돌아오는 길에, 소년은 혼잣속으로, 소녀가 이사를 간다는 말을 수없이 되
뇌어 보았다. 무어 그리 안타까울 것도 서러울 것도 없었다. 그렇건만, 소년은 지금 자기가 
씹고 있는 대추알의 단맛을 모르고 있었다.
이 날 밤, 소년은 몰래 덕쇠 할아버지네 호두밭으로 갔다.
낯에 봐 두었던 나무로 올라갔다. 그리고, 봐 두었던 가지를 향해 작대기를 내리쳤다. 호두
송이 떨어지는 소리가 별나게 크게 들렸다. 가슴이 선뜩했다. 그러나 다음 순간, 굵은 호두
야 많이 떨어져라, 많이 떨어져라, 저도 모를 힘에 이끌려 마구 작대기를 내리 치는 것이
었다.
돌아오는 길에는 열 이틀 달이 지우는 그늘만 골라 디뎠다. 그늘의 고마움을 처음 느꼈다.
불룩한 주머니를 어루만졌다. 호두송이를 맨손으로 깠다가는 옴이 오르기 쉽다는 말 같은 건 아
무렇지도 않았다. 그저 근동에서 제일 가는 이 덕쇠 할아버지네 호두를 어서 소녀에게 맛보여야 
한다는 생각만이 앞섰다.
그러다, 아차 하는 생각이 들었다. 소녀더러 병이 좀 낫거들랑 이사 가기 전에 한 번 개울
가로 나와 달라는 말을 못해 둔 것이었다. 바보 같은것, 바보 같은것.
이튿날, 소년이 학교에서 돌아오니, 아버지가 나들이옷으로 갈아입고 닭 한 마리를 안고 
있었다.
어디 가시느냐고 물었다.
그 말에도 대꾸도 없이, 아버지는 안고 있는 닭의 무게를 겨냥해 보면서,
“이만하면 될까?”
어머니가 망태기를 내주며,
“벌써 며칠째 ‘걀걀’하고 알 날 자리를 보던데요. 크진 않아도살은 쪘을 거

**[벡터저장소]** Chroma DB

In [ ]:
# 벡터저장소(Chroma)
#!pip install -q chromadb
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

llm = ChatOpenAI(temperature=0, model_name='gpt-4o-mini')
embedding_model = OpenAIEmbeddings()
db_dir = "chroma/"
doc_file_path = "소나기 - 황순원.pdf"

if(os.path.isdir(db_dir) == False):
    document = PyMuPDFLoader(doc_file_path).load()

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
    )
    split_docs = splitter.split_documents(document)
    print(f'총 분할된 도큐먼트 수: {len(split_docs)}')

    vs = Chroma.from_documents(
        split_docs,
        embedding_model,
        persist_directory = db_dir,
        #collection_metadata = {'hnsw:space': 'cosine'}, # 유사도 계산에 코사인 유사도를 사용
    )
    print("saved")
else:
    vs = Chroma(persist_directory=db_dir, embedding_function=embedding_model)
    print('loaded')

query = "소녀는 왜 이사를 오게 되었는가?"
docs = vs.similarity_search(query, k=2)
for index, doc in enumerate(docs, start=1):
    print(f'[{index}]\n' + doc.page_content+ "\n========\n")

총 분할된 도큐먼트 수: 13
saved
[1]
- 6 -
소년은 소녀네가 이사해 오기 전에 벌써 어른들의 이야기를 들어서, 윤 초시 손자(孫子)가 
서울서 사업에 실패해 가지고 고향에 돌아오지 않을 수 없게 되었다는 걸 알고 있었다. 그
것이 이번에는 고향 집마저 남의 손에 넘기게 된 모양이었다.
“왜 그런지 난 이사 가는 게 싫어졌다. 어른들이 하는 일이니 어쩔 수 없지만…….”
전에 없이, 소녀의 까만 눈에 쓸쓸한 빛이 떠돌았다.
소녀와 헤어져 돌아오는 길에, 소년은 혼잣속으로, 소녀가 이사를 간다는 말을 수없이 되
뇌어 보았다. 무어 그리 안타까울 것도 서러울 것도 없었다. 그렇건만, 소년은 지금 자기가 
씹고 있는 대추알의 단맛을 모르고 있었다.
이 날 밤, 소년은 몰래 덕쇠 할아버지네 호두밭으로 갔다.
낯에 봐 두었던 나무로 올라갔다. 그리고, 봐 두었던 가지를 향해 작대기를 내리쳤다. 호두
송이 떨어지는 소리가 별나게 크게 들렸다. 가슴이 선뜩했다. 그러나 다음 순간, 굵은 호두
야 많이 떨어져라, 많이 떨어져라, 저도 모를 힘에 이끌려 마구 작대기를 내리 치는 것이
었다.
돌아오는 길에는 열 이틀 달이 지우는 그늘만 골라 디뎠다. 그늘의 고마움을 처음 느꼈다.
불룩한 주머니를 어루만졌다. 호두송이를 맨손으로 깠다가는 옴이 오르기 쉽다는 말 같은 건 아
무렇지도 않았다. 그저 근동에서 제일 가는 이 덕쇠 할아버지네 호두를 어서 소녀에게 맛보여야 
한다는 생각만이 앞섰다.
그러다, 아차 하는 생각이 들었다. 소녀더러 병이 좀 낫거들랑 이사 가기 전에 한 번 개울
가로 나와 달라는 말을 못해 둔 것이었다. 바보 같은것, 바보 같은것.
이튿날, 소년이 학교에서 돌아오니, 아버지가 나들이옷으로 갈아입고 닭 한 마리를 안고 
있었다.
어디 가시느냐고 물었다.
그 말에도 대꾸도 없이, 아버지는 안고 있는 닭의 무게를 겨냥해 보면서,
“이만하면 될까?”
어머니가 망태기를 내주며,
“벌써 며칠째 ‘걀걀’하고 알 날 자리를 보던데요. 크진 않아도살은 쪘을 거여요.

## **4) Retrieval Chain**

**RetrievalQA** (chain_type="stuff")

In [ ]:
# RetrievalQA (chain_type="stuff")
# FAISS를 사용했으며 임베딩 결과 저장하지 않음.
from langchain_openai import ChatOpenAI
from langchain.document_loaders import PyMuPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

llm = ChatOpenAI(temperature=0, model_name='gpt-4o-mini')
embeddings = OpenAIEmbeddings()
doc_file_path = "소나기 - 황순원.pdf"

loader = PyMuPDFLoader(doc_file_path)
raw_doc = loader.load()
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)

split_docs = splitter.split_documents(raw_doc)
print(f'총 분할된 도큐먼트 수: {len(split_docs)}')
db = FAISS.from_documents(split_docs, embeddings)

retriever = db.as_retriever()

from langchain.chains import RetrievalQA
retriever = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever)

query = "소녀는 왜 이사를 오게 되었는가?"
response = retriever.invoke({"query": query})
print(response['result'])

# RetrievalQA 체인의 실행과정을 모니터링하려면?

총 분할된 도큐먼트 수: 13


> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
- 6 -
소년은 소녀네가 이사해 오기 전에 벌써 어른들의 이야기를 들어서, 윤 초시 손자(孫子)가 
서울서 사업에 실패해 가지고 고향에 돌아오지 않을 수 없게 되었다는 걸 알고 있었다. 그
것이 이번에는 고향 집마저 남의 손에 넘기게 된 모양이었다.
“왜 그런지 난 이사 가는 게 싫어졌다. 어른들이 하는 일이니 어쩔 수 없지만…….”
전에 없이, 소녀의 까만 눈에 쓸쓸한 빛이 떠돌았다.
소녀와 헤어져 돌아오는 길에, 소년은 혼잣속으로, 소녀가 이사를 간다는 말을 수없이 되
뇌어 보았다. 무어 그리 안타까울 것도 서러울 것도 없었다. 그렇건만, 소년은 지금 자기가 
씹고 있는 대추알의 단맛을 모르고 있었다.
이 날 밤, 소년은 몰래 덕쇠 할아버지네 호두밭으로 갔다.
낯에 봐 두었던 나무로 올라갔다. 그리고, 봐 두었던 가지를 향해 작대기를 내리쳤다. 호두
송이 떨어지는 소리가 별나게 크게 들렸다. 가슴이 선뜩했다. 그러나 다음 순간, 굵은 호두
야 많이 떨어져라, 많이 떨어져라, 저도 모를 힘에 이끌려 마구 작대기를 내리 치는 것이
었다.
돌아오는 길에는 열 이틀 달이 지우는 그늘만 골라 디뎠다. 그늘의 고마움을 처음 느꼈다.
불룩한 주머니를 어루만졌다. 호두송이를 맨손으로 깠다가는 옴이 오르기 쉽다는 말 같은 건 아
무렇지도 않았다. 그저 근동에서 제

**로그 활성화**

In [ ]:
import langchain
langchain.debug = False
langchain.verbose = False

# langchain.verbose 를 True로 설정하여 프롬프트와 retrieval이 반환한 문서 확인 가능

**RetrievalQA** (chain_type="map_rerank")

In [ ]:
# RetrievalQA (chain_type="map_rerank")
# FAISS를 사용했으며 임베딩 결과 저장하지 않음
from langchain_openai import ChatOpenAI
from langchain.document_loaders import PyMuPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

llm = ChatOpenAI(temperature=0, model_name='gpt-4o')
embeddings = OpenAIEmbeddings()
doc_file_path = "소나기 - 황순원.pdf"

loader = PyMuPDFLoader(doc_file_path)
raw_doc = loader.load()
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)

split_docs = splitter.split_documents(raw_doc)
print(f'총 분할된 도큐먼트 수: {len(split_docs)}')
db = FAISS.from_documents(split_docs, embeddings)

retriever = db.as_retriever(search_type="mmr", search_kwargs={"k": 5})

from langchain.chains import RetrievalQA
retriever = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_rerank",
    retriever=retriever)

query = "소녀가 대추를 딴 이유는?"
response = retriever.invoke({"query": query})
print(response['result'])

# 정신없는 로그, 도저히 정리가 안된다.

총 분할된 도큐먼트 수: 13


> Entering new RetrievalQA chain...


> Entering new MapRerankDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

In addition to giving an answer, also return a score of how fully it answered the user's question. This should be in the following format:

Question: [question here]
Helpful Answer: [answer here]
Score: [score between 0 and 100]

How to determine the score:
- Higher is a better answer
- Better responds fully to the asked question, with sufficient level of detail
- If you do not know the answer based on the context, that should be a score of 0
- Don't be overconfident!

Example #1

Context:
---------
Apples are red
---------
Question: what color are apples?
Helpful Answer: red
Score: 100

Example #2

Context:
---------
it was night and the witness forgot his glass

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:369: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(



> Finished chain.

> Finished chain.

> Finished chain.
Helpful Answer: 윤 초시 손자가 서울에서 사업에 실패해 고향으로 돌아오게 되었기 때문이다.


'query = "소녀가 대추를 딴 이유는?"\nresponse = retriever.invoke({"query": query})\nprint(response[\'result\'])'

In [ ]:
# LangSmith 를 이용한 모니터링
os.environ["LANGCHAIN_PROJECT"] = "chain-monitor"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"]="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" # <= 자신의 API키로 바꾸세요
os.environ["LANGCHAIN_TRACING_V2"]="true"  # true 이면 계속 모니터링 됩니다. 그만 하려면 false 로 바꾸세요

In [ ]:
# LangSmith 를 이용한 모니터링
# RetrievalQA (chain_type="stuff")
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

llm = ChatOpenAI(temperature=0, model_name='gpt-4o-mini')
embeddings = OpenAIEmbeddings()
doc_file_path = "소나기 - 황순원.pdf"

loader = PyMuPDFLoader(doc_file_path)
raw_doc = loader.load()
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)

split_docs = splitter.split_documents(raw_doc)
print(f'총 분할된 도큐먼트 수: {len(split_docs)}')
db = FAISS.from_documents(split_docs, embeddings)

retriever = db.as_retriever(search_type="mmr", search_kwargs={"k": 5}) # <= k값의 중요성

from langchain.chains import RetrievalQA
retriever = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever)

query = "소설 끝에서 소녀는 어떠한 결말을 맞이한거야?"
response = retriever.invoke({"query": query})
print(response['result'])

총 분할된 도큐먼트 수: 13
소설의 끝에서 소녀는 아프고, 결국 죽음을 맞이하게 됩니다. 그녀의 가족은 그녀가 어린 나이에 잃어버린 것에 대해 안타까워하며, 그녀의 마지막 소원인 입던 옷을 그대로 입혀서 묻어 달라는 이야기를 나누는 장면이 있습니다.


**프롬프트 변형과 관련한 실습**

In [ ]:
# 보너스: 검증된 프롬프트 가져오기
from langchain import hub
import pprint

prompt = hub.pull("rlm/rag-prompt", api_url="https://api.hub.langchain.com")

# 프롬프트 내용 출력
pprint.pp(prompt.messages[0].prompt.__dict__, indent=4)

{   'name': None,
    'input_variables': ['context', 'question'],
    'optional_variables': [],
    'input_types': {},
    'output_parser': None,
    'partial_variables': {},
    'metadata': None,
    'tags': None,
    'template': 'You are an assistant for question-answering tasks. Use the '
                'following pieces of retrieved context to answer the question. '
                "If you don't know the answer, just say that you don't know. "
                'Use three sentences maximum and keep the answer concise.\n'
                'Question: {question} \n'
                'Context: {context} \n'
                'Answer:',
    'template_format': 'f-string',
    'validate_template': False}


In [ ]:
# LCEL(LangChain Expression Language) 버전1
# rlm/rag-prompt 프롬프트 사용
from operator import itemgetter

from langchain.chat_models import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain import hub

llm = ChatOpenAI(temperature=0, model_name='gpt-4o-mini')
embeddings = OpenAIEmbeddings()
doc_file_path = "소나기 - 황순원.pdf"

loader = PyMuPDFLoader(doc_file_path)
raw_doc = loader.load()
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)

split_docs = splitter.split_documents(raw_doc)
print(f'총 분할된 도큐먼트 수: {len(split_docs)}')
db = FAISS.from_documents(split_docs, embeddings)
print(f"{db.distance_strategy}\n")

retriever = db.as_retriever(search_type="mmr", search_kwargs={"k": 5})

prompt = hub.pull("rlm/rag-prompt")  # langchainhub에서 검증된 prompt를 가져옴.

model = ChatOpenAI(model_name='gpt-4o-mini', temperature=0)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

query = "소설 끝에서 소녀는 어떠한 결말을 맞이한거야? 근거를 찾아 설명해줘"
response = chain.invoke(query)
print(response)

총 분할된 도큐먼트 수: 13
EUCLIDEAN_DISTANCE

소설 "소나기"에서 소녀는 결국 죽음을 맞이하게 됩니다. 그녀는 아프고, 죽기 전에 자신의 입던 옷을 그대로 입혀서 묻어 달라고 부탁하는 장면이 있습니다. 이는 소녀의 비극적인 결말을 암시하며, 그녀의 삶이 짧고 잔망스러웠음을 보여줍니다.


In [ ]:
# LCEL(LangChain Expression Language) 버전2
# Prompt를 직접 설계하는 방법
from operator import itemgetter

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.callbacks.tracers import ConsoleCallbackHandler

llm = ChatOpenAI(temperature=0, model_name='gpt-4o-mini')
embeddings = OpenAIEmbeddings()
doc_file_path = "소나기 - 황순원.pdf"

loader = PyMuPDFLoader(doc_file_path)
raw_doc = loader.load()
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)

split_docs = splitter.split_documents(raw_doc)
print(f'총 분할된 도큐먼트 수: {len(split_docs)}')
db = FAISS.from_documents(split_docs, embeddings)
print(f"{db.distance_strategy}\n")

retriever = db.as_retriever(search_type="mmr", search_kwargs={"k": 5})

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(model_name='gpt-4o', temperature=0)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

print(chain.invoke("소설 끝에서 소녀가 어떠한 결말을 맞이한거야? 근거를 찾아 설명해줘"))

총 분할된 도큐먼트 수: 13
EUCLIDEAN_DISTANCE

소설의 끝에서 소녀는 죽음을 맞이합니다. 이 결말은 다음과 같은 문장들에서 확인할 수 있습니다:

1. "윤 초시 댁도 말이 아니야, 그 많던 전답을 다 팔아 버리고, 대대로 살아오던 집마저 남의 손에 넘기더니, 또 악상까지 당하는걸 보면……."
2. "증손(曾孫)이라곤 계집애 그 애 하나뿐이었지요?"
3. "그렇지, 사내 애 둘 있던 건 어려서 잃어버리고……."
4. "글쎄 말이지. 이번 앤 꽤 여러 날 앓는 걸 약도 변변히 못써 봤다더군. 지금 같아서 윤 초시네도 대가 끊긴 셈이지.……그런데참, 이번 계집앤 어린 것이 여간 잔망스럽지가 않아. 글쎄, 죽기전에 이런 말을 했다지 않아? 자기가 죽거든 자기 입던 옷을 꼭그대로 입혀서 묻어 달라고……."

이 문장들에서 소녀가 병으로 여러 날 앓다가 결국 죽음을 맞이했음을 알 수 있습니다. 특히, "죽기전에 이런 말을 했다지 않아? 자기가 죽거든 자기 입던 옷을 꼭그대로 입혀서 묻어 달라고……."라는 문장은 소녀가 죽기 전에 자신의 장례에 대해 언급한 것을 나타내고 있습니다.


# **[LangChain]　5. Agent**

Agent의 작동원리 ReAct 개념 예제

In [ ]:
# ReAct: Synergizing Reasoning and Acting in Language Models 논문에 소개한 메커너즘으로 작동
# (https://react-lm.github.io)
import langchain
langchain.verbose = True

import pprint
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent, load_tools
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
tools = load_tools(["terminal"], allow_dangerous_tools=True)

prompt = hub.pull("hwchase17/react")
pprint.pp(prompt.template, indent=4, width=100)  # <=

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

result = agent_executor.invoke({"input": "sample_data 디렉터리에 있는 파일 목록을 알려줘"})
print(result["output"])

('Answer the following questions as best you can. You have access to the following tools:\n'
 '\n'
 '{tools}\n'
 '\n'
 'Use the following format:\n'
 '\n'
 'Question: the input question you must answer\n'
 'Thought: you should always think about what to do\n'
 'Action: the action to take, should be one of [{tool_names}]\n'
 'Action Input: the input to the action\n'
 'Observation: the result of the action\n'
 '... (this Thought/Action/Action Input/Observation can repeat N times)\n'
 'Thought: I now know the final answer\n'
 'Final Answer: the final answer to the original input question\n'
 '\n'
 'Begin!\n'
 '\n'
 'Question: {input}\n'
 'Thought:{agent_scratchpad}')


> Entering new AgentExecutor chain...
I need to check the contents of the "sample_data" directory to list the files it contains. I'll use the terminal to do this.

Action: terminal  
Action Input: ls sample_data  Executing command:
 ls sample_data
anscombe.json
california_housing_test.csv
california_housing_train.csv
mnist_

/usr/local/lib/python3.10/dist-packages/langchain_community/tools/shell/tool.py:32: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


I now know the final answer, which is the list of files in the "sample_data" directory.  
Final Answer: The files in the "sample_data" directory are: anscombe.json, california_housing_test.csv, california_housing_train.csv, mnist_test.csv, mnist_train_small.csv, README.md.

> Finished chain.
The files in the "sample_data" directory are: anscombe.json, california_housing_test.csv, california_housing_train.csv, mnist_test.csv, mnist_train_small.csv, README.md.


In [ ]:
# openai_functions_agent (Deprecated)
from langchain.agents import load_tools
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
tools = load_tools(["ddg-search"])
prompt = hub.pull("hwchase17/openai-functions-agent")

agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

result = agent_executor.invoke({"input": "서울과 부산의 날씨를 알려줘"})
print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_search` with `{'query': '서울 부산 날씨'}`


날씨, 2024년 장마기간→'서울·수도권 80mm비' 3호태풍 개미 예상경로. 주요 기사. 날씨, 3호 태풍영향→'34도 폭염' 2024년 장마기간→서울 80mm비 예상경로 [속보] 부산날씨→호우주의보, 온천천산책로 '전면통제' 천둥번개 ... 부산 주간 날씨 예보 / 네이버 도로공사는 이번 주말, 일부 지역 눈, 비 예보에 도로 살얼음 주의를 당부했다. 특히 교량과 터널 입·출구, 음지 커브길은 미끄럼 사고에 취약한 구간이므로 급제동과 급가속은 삼가고 제동 시 브레이크를 나눠 밟는 것이 좋겠다. (서울=연합뉴스) 박형빈 기자 = 일요일인 5일은 전국이 흐리고 비가 오겠다. ... 날씨 역주행 (부산=연합뉴스) 손형주 기자 = 늦더위가 기승을 부리고 있는 3일 부산 해운대구 송정해수욕장에서 시민들이 해안가에서 더위를 식히고 있다. 2023.11.3 handbrother@yna.co.kr ... 이번주 '주말날씨' 예보에 관심이 쏠리고 있다. 19일 기상청은 전국·서울·부산·대구·제주 주말날씨를 발표했다.20일 주말 아침부터 제주와 전남을 ... [날씨] 설 연휴까지 먼지↑…낮 서울 4℃ · 부산 9℃ ... 서울특별시 양천구 목동서로 161; 고객센터 : 1577-1003; Email : newsservice@sbs.co.kr;현재 서울과 부산의 날씨는 다음과 같습니다:

- **서울**: 흐리고 비가 오는 날씨입니다. 기온은 낮에 약 4도 정도로 예상됩니다.
- **부산**: 호우주의보가 발효 중이며, 기온은 낮에 약 9도 정도로 예상됩니다. 

추가적으로, 부산에서는 천둥과 번개가 동반될 수 있으니 주의가 필요합니다.

> Finished chain.
현재 서울과 부산의 날씨는 다음과 같습니다:

- **서울**: 흐리고 비가 오는 날씨입니다. 기온은 낮에 약 4도 정도로 예상

In [ ]:
# openai_tools_agent
from langchain.agents import load_tools
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
tools = load_tools(["ddg-search"])
prompt = hub.pull("hwchase17/openai-tools-agent")

agent = create_openai_tools_agent(llm, tools, prompt)  # <=
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

result = agent_executor.invoke({"input": "지금 이 시각 서울과 부산의 날씨를 알려줘"})
print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_search` with `{'query': 'current weather in Seoul'}`


Seoul Weather Forecasts. Weather Underground provides local & long-range weather forecasts, weatherreports, maps & tropical weather conditions for the Seoul area. Fri. Updated 7/24 9:47 AM. The hourly local weather forecast shows hour by hour weather conditions like temperature, feels like temperature, humidity, amount of precipitation and chance of precipitation, wind and gusts for Seoul. Anyang-si. 91 °F / 79 °F. Uijeongbu-si. 91 °F / 78 °F. Gwangmyeong. 90 °F / 77 °F. Today's and tonight's professional weather forecast for Seoul. Precipitation radar, HD satellite images, and current weather warnings, hourly temperature, chance of rain, and sunshine hours. Weather Today Weather Hourly 14 Day Forecast Yesterday/Past Weather Climate (Averages) Now. 70 °F. Clear. Feels Like: 70 °F Forecast: 74 / 64 °F Wind: 5 mph ↑ from West. Location: Seoul / Kimp'O International Airpor

In [ ]:
# 에이전트/툴
#!pip install -q google-search-results numexpr
from langchain_openai import ChatOpenAI
from langchain import hub
llm = ChatOpenAI(temperature=0, model_name='gpt-4o')

from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.agents import create_openai_tools_agent, AgentExecutor

tools = load_tools(["ddg-search", "wikipedia"], llm=llm)   # 도구 추가함

prompt = hub.pull("hwchase17/openai-tools-agent")
agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

result=agent_executor.invoke({"input": "너는 누구니?"})
print(result['output'])

result=agent_executor.invoke({"input": "사람도 동물이야?"})
print(result['output'])

result=agent_executor.invoke({"input": "4옥타브 '도'음의 주파수는 440*2^(3/12) 입니다. 구체적인 수치는?"})
print(result['output'])

result=agent_executor.invoke({"input": "라인야후의 소유주는 누구?"})
print(result['output'])

result=agent_executor.invoke({"input": "연이율 6%일 때 한달에 얼마씩 저금해야 10년만에 1억을 만들수 있을까?"})
print(result['output'])

result=agent_executor.invoke({"input": "2024년도 현재 에드 시런은 몇 살?"})
print(result['output'])



> Entering new AgentExecutor chain...
저는 여러분의 질문에 답하고 다양한 작업을 도와드리는 인공지능 어시스턴트입니다. 무엇을 도와드릴까요?

> Finished chain.
저는 여러분의 질문에 답하고 다양한 작업을 도와드리는 인공지능 어시스턴트입니다. 무엇을 도와드릴까요?


> Entering new AgentExecutor chain...
네, 사람도 동물입니다. 사람은 동물계에 속하는 포유류입니다. 생물학적으로 사람은 영장류에 속하며, 특히 호미니드과에 속하는 종입니다. 따라서 사람도 동물의 일종으로 분류됩니다.

> Finished chain.
네, 사람도 동물입니다. 사람은 동물계에 속하는 포유류입니다. 생물학적으로 사람은 영장류에 속하며, 특히 호미니드과에 속하는 종입니다. 따라서 사람도 동물의 일종으로 분류됩니다.


> Entering new AgentExecutor chain...
4옥타브 '도'음의 주파수를 구하기 위해 주어진 공식을 사용하면 됩니다. 주어진 공식은:

\[ f = 440 \times 2^{\frac{3}{12}} \]

이 공식을 계산해 보겠습니다.

\[ 2^{\frac{3}{12}} = 2^{0.25} \approx 1.189207 \]

따라서,

\[ f = 440 \times 1.189207 \approx 523.251 \]

따라서, 4옥타브 '도'음의 주파수는 약 523.251 Hz입니다.

> Finished chain.
4옥타브 '도'음의 주파수를 구하기 위해 주어진 공식을 사용하면 됩니다. 주어진 공식은:

\[ f = 440 \times 2^{\frac{3}{12}} \]

이 공식을 계산해 보겠습니다.

\[ 2^{\frac{3}{12}} = 2^{0.25} \approx 1.189207 \]

따라서,

\[ f = 440 \times 1.189207 \approx 523.251 \]

따라서, 4옥타브 '도'음의 주파수는 약 523.251 Hz입니다.


> Ent

In [ ]:
# 과거 방식임. 쓰지마!
# PythonREPL 도구에 버그 있음. 명시적으로 print() 하지 않으면 LMM이 못 알아먹음..
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent, load_tools
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_experimental.utilities import PythonREPL
from langchain.agents import Tool, load_tools, create_react_agent, AgentExecutor

llm = ChatOpenAI(model_name='gpt-4o', temperature=0, streaming=True)

tavily_tool = TavilySearchResults(k=5)

python_repl = PythonREPL()
python_repl_tool = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. \
    Input should be a valid python command. \
    If you want to see the output of a value, you should print it out with `print(...)`.",
    func=python_repl.run,
)

tools = [tavily_tool, python_repl_tool]

prompt = hub.pull("hwchase17/react")

agent = create_react_agent(llm, tools, prompt)

cbm = ConversationBufferMemory(
				memory_key='chat_history',
				return_messages=True)

agent_executor = AgentExecutor(
    agent=agent, tools=tools,
    memory=cbm,
    get_chat_history=lambda h : h,
    handle_parsing_errors=True, verbose=True)
"""
result=agent_executor.invoke({"input": "너는 누구니?"})
print(result['output'])

result=agent_executor.invoke({"input": "사람도 동물이야?"})
print(result)
"""
result=agent_executor.invoke({"input": "4옥타브 '도'음의 주파수는 440*2^(3/12) 입니다. 구체적인 수치는?"})
print(result)

result=agent_executor.invoke({"input": "라인야후의 소유주는 누구?"})
print(result)

result=agent_executor.invoke({"input": "연 복리 6%일 때 한달에 얼마씩 저금해야 10년만에 1억을 만들수 있을까?"})
print(result)

result=agent_executor.invoke({"input": "2024년도 현재 에드 시런은 몇 살?"})
print(result)




> Entering new AgentExecutor chain...
To find the specific frequency of the 4-octave 'C' note, given the formula \( 440 \times 2^{3/12} \), I need to perform the calculation.

Action: python_repl
Action Input: 
```python
frequency = 440 * 2**(3/12)
print(frequency)
```523.2511306011972
I now know the final answer.

Final Answer: 4옥타브 '도'음의 주파수는 약 523.25 Hz입니다.

> Finished chain.
{'input': "4옥타브 '도'음의 주파수는 440*2^(3/12) 입니다. 구체적인 수치는?", 'chat_history': [HumanMessage(content="4옥타브 '도'음의 주파수는 440*2^(3/12) 입니다. 구체적인 수치는?"), AIMessage(content="4옥타브 '도'음의 주파수는 약 523.25 Hz입니다.")], 'output': "4옥타브 '도'음의 주파수는 약 523.25 Hz입니다."}


> Entering new AgentExecutor chain...
라인야후의 소유주를 알아내기 위해 검색을 해야 합니다.
Action: tavily_search_results_json
Action Input: "라인야후 소유주"[{'url': 'https://news.sbs.co.kr/news/endPage.do?news_id=N1007730668', 'content': '네이버 "라인야후가 보안 강화 주체" 입장 정부에 전달. 라인야후 대주주 소프트뱅크가 라인야후 자본관계 재검토를 단기적으로는 단념한 것으로 ...'}, {'url': 'https://economist.co.kr/article/view/ecn202405100023', 'content': 

**사용자 정의 Tool 예시 #1**

In [ ]:
from langchain.tools import Tool
from langchain.agents import load_tools, create_react_agent, AgentExecutor

def Komifoma_func(param):
        return "박정진"

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

tools = [
    Tool.from_function(
        func=Komifoma_func,
        name="Komifoma_Answer",
        description="Komifoma World에 대한 모든 질문에 대한 답"
    ),
]

prompt = hub.pull("hwchase17/react")

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
result = agent_executor.invoke({"input": "Komifoma를 누가 만들었나요?"})
result = agent_executor.invoke({"input": "Komifoma를 누가 끝장냈나요?"})



> Entering new AgentExecutor chain...
I need to find out who created Komifoma. 
Action: Komifoma_Answer
Action Input: "Komifoma를 누가 만들었나요?"박정진I now know the final answer.  
Final Answer: Komifoma는 박정진에 의해 만들어졌습니다.

> Finished chain.


> Entering new AgentExecutor chain...
I need to find out who ended Komifoma. 
Action: Komifoma_Answer
Action Input: "Komifoma를 누가 끝장냈나요?"박정진I now know the final answer.  
Final Answer: 박정진

> Finished chain.


**사용자 정의 Tool 예시 #2**

In [ ]:
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.tools import Tool
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langchain.agents import AgentExecutor, create_openai_functions_agent

class WeddingInput(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")

def concat(a: int, b: int) -> int:
    """두 수의 결혼"""
    return f"{a}{b}"


calculator = StructuredTool.from_function(
    func=concat,
    name="ConCat",
    description="두 수의 결혼 결과",
    args_schema=WeddingInput,
    return_direct=True,
)

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
prompt = hub.pull("hwchase17/openai-functions-agent")
tools = [calculator]

print(calculator.name)
print(calculator.description)
print(calculator.args)

agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
result = agent_executor.invoke({"input": "12와 23이 결혼하면?"})

ConCat
두 수의 결혼 결과
{'a': {'title': 'A', 'description': 'first number', 'type': 'integer'}, 'b': {'title': 'B', 'description': 'second number', 'type': 'integer'}}


> Entering new AgentExecutor chain...

Invoking: `ConCat` with `{'a': 12, 'b': 23}`


1223


> Finished chain.


**사용자 정의 Tool 예시 #3**

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain import LLMChain
from langchain.agents import Tool, load_tools, create_react_agent, AgentExecutor

gpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

summarize_template = """아래의 글을 단 한 문장으로 요약해 주세요.

{input}
"""
summarize_prompt = PromptTemplate(
   input_variables=["input"],
   template=summarize_template,
)
summarize_chain = LLMChain(llm=gpt, prompt=summarize_prompt)

tools = [
    Tool.from_function(
        func=summarize_chain.run,
        name="Summarizer",
        description="Text summarizer"
    ),
]

prompt = hub.pull("hwchase17/react")

agent = create_react_agent(gpt, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

text = """다음을 요약해 주세요.

안녕하세요! 저는 ChatGPT라고 불리는 AI 언어 모델입니다. OpenAI가 개발한 GPT-3.5 아키텍처를 기반으로 합니다. 저는 자연어 이해와 생성을 전문으로 하며, 다양한 주제에 대한 질문에 답하거나, 대화를 나누는 것을 잘합니다.
제 트레이닝 데이터는 2021년 9월까지의 정보를 기반으로 하기 때문에, 그 이후의 사건에 대해서는 지식이 없습니다. 하지만, 가능한 한 도움을 드리기 위해 노력할 것입니다.
질문이나 대화, 정보 공유 등, 어떤 도움이든 편하게 말씀해 주세요! 잘 부탁드립니다."""

result = agent_executor.invoke({"input": text})
print(result['output'])



> Entering new AgentExecutor chain...
이 내용을 요약하기 위해서는 주요 정보를 간추려야 합니다. 이 텍스트는 ChatGPT의 소개와 기능, 그리고 트레이닝 데이터의 한계를 설명하고 있습니다. 

Action: Summarizer  
Action Input: "안녕하세요! 저는 ChatGPT라고 불리는 AI 언어 모델입니다. OpenAI가 개발한 GPT-3.5 아키텍처를 기반으로 합니다. 저는 자연어 이해와 생성을 전문으로 하며, 다양한 주제에 대한 질문에 답하거나, 대화를 나누는 것을 잘합니다. 제 트레이닝 데이터는 2021년 9월까지의 정보를 기반으로 하기 때문에, 그 이후의 사건에 대해서는 지식이 없습니다. 하지만, 가능한 한 도움을 드리기 위해 노력할 것입니다. 질문이나 대화, 정보 공유 등, 어떤 도움이든 편하게 말씀해 주세요! 잘 부탁드립니다."  저는 OpenAI가 개발한 AI 언어 모델 ChatGPT로, 자연어 이해와 생성에 특화되어 있으며, 2021년 9월까지의 정보를 바탕으로 다양한 질문에 답변하고 대화를 나눌 수 있습니다.이제 최종 답변을 알게 되었습니다.  
Final Answer: 저는 OpenAI가 개발한 AI 언어 모델 ChatGPT로, 자연어 이해와 생성에 특화되어 있으며, 2021년 9월까지의 정보를 바탕으로 다양한 질문에 답변하고 대화를 나눌 수 있습니다.

> Finished chain.
저는 OpenAI가 개발한 AI 언어 모델 ChatGPT로, 자연어 이해와 생성에 특화되어 있으며, 2021년 9월까지의 정보를 바탕으로 다양한 질문에 답변하고 대화를 나눌 수 있습니다.
